# To do

* 綺麗な可視化
* 記事にする

In [112]:
import importlib

import torch
import numpy as np
import pandas as pd
import plotly.graph_objects as go

torch.set_default_dtype(torch.float64)

import linalg_utils
importlib.reload(linalg_utils)

import misc_utils
importlib.reload(misc_utils)

import gp_utils
importlib.reload(gp_utils)

import gp_regression
importlib.reload(gp_regression)
from gp_regression import GP_regression
from gp_regression import Sparse_GP_regression_SoR 
from gp_regression import Sparse_GP_regression_DTC
from gp_regression import Sparse_GP_regression_FITC

# 実験


## 準備

In [28]:
class Latent_Function():
    def __init__(self, noise_level=0.1):
        self.noise_level = noise_level
        self.noise = None

    def _f(self, X):
        # ここを好きにカスタマイズ
        tmp = np.sin(X) + np.cos(X)
        return tmp
    
    def f(self, X, observed=False):
        tmp = self._f(X)

        if observed is True:
            self.noise = np.random.rand(*X.shape) * self.noise_level

            if isinstance(X, np.ndarray):
                return np.array(tmp) + self.noise
            elif isinstance(X, torch.Tensor):
                return tmp + torch.Tensor(self.noise)
        
        else:
            if isinstance(X, np.ndarray):
                return np.array(tmp)
            elif isinstance(X, torch.Tensor):
                return tmp


def make_data(f, X=None, X_pred=None):
    if X is None:
        X_normal = torch.randn(50) * 1.5 + 4 
        X_normal = torch.clip(X_normal, 0, 10) 
        X_uniform = torch.rand(50) * 6 + 2  
        X_combined = torch.cat([X_normal, X_uniform])
        X, _ = torch.sort(X_combined)
        X = X.reshape(-1, 1)

    y = f(X, observed=True)

    if X_pred is None:
        X_max, X_min = X.max(), X.min()
        interval = torch.abs(X_max - X_min)
        size = int(torch.ceil(interval / 0.1))
        X_pred_max, X_pred_min = X_max + interval * 0.2, X_min - interval * 0.2
        X_pred = torch.linspace(X_pred_min, X_pred_max, size).reshape(-1, 1)

    return X, y, X_pred


def visualize(X_pred, f, mean, cov, p_inputs=None, title=""):
    mean = mean.ravel().detach().numpy()
    var = torch.diagonal(cov).detach().numpy()
    X_pred = X_pred.ravel().detach().numpy()

    if p_inputs is not None:
        p_inputs = p_inputs.ravel().detach().numpy()

    credible_interval = 1.96 * np.sqrt(var) # 95%

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=X_pred, y=f(X_pred), line_color="#00CC00", name="Latent Function"))
    fig.add_trace(go.Scatter(x=X_pred, y=mean, line_color="#FE73FF", name="Mean"))
    fig.add_trace(go.Scatter(x=X_pred, y=mean-credible_interval, mode='lines', line=dict(color='lightgray'), showlegend=False))
    fig.add_trace(go.Scatter(x=X_pred, y=mean+credible_interval, mode='lines', line=dict(color='lightgray'), fill='tonexty', showlegend=False))
    
    if p_inputs is not None:
        fig.add_trace(go.Scatter(x=p_inputs, y=f(p_inputs), mode='markers', marker=dict(size=8, color="#FF0000"), name="Pseudo-inputs"))

    fig.update_layout(title=title, xaxis_title="X", yaxis_title="f*")
    fig.show()

In [29]:
f = Latent_Function(noise_level=0.1).f
X, y, X_pred = make_data(f)

## GP regression

In [95]:
model1 = GP_regression(X, y)
mean1, cov1 = model1.predict(X_pred.clone())

# visualize(X_pred, f, mean1, cov1, title="GP_regression")

In [96]:
model1.optimize(iteration=200, learning_rate=0.01)
mean1_opt, cov1_opt = model1.predict(X_pred.clone())

# visualize(X_pred, f, mean1_opt, cov1_opt, title="Optimized GP_regression")

opt_iter: 1/200
opt_iter: 2/200
opt_iter: 3/200
opt_iter: 4/200
opt_iter: 5/200
opt_iter: 6/200
opt_iter: 7/200
opt_iter: 8/200
opt_iter: 9/200
opt_iter: 10/200
opt_iter: 11/200
opt_iter: 12/200
opt_iter: 13/200
opt_iter: 14/200
opt_iter: 15/200
opt_iter: 16/200
opt_iter: 17/200
opt_iter: 18/200
opt_iter: 19/200
opt_iter: 20/200
opt_iter: 21/200
opt_iter: 22/200
opt_iter: 23/200
opt_iter: 24/200
opt_iter: 25/200
opt_iter: 26/200
opt_iter: 27/200
opt_iter: 28/200
opt_iter: 29/200
opt_iter: 30/200
opt_iter: 31/200
opt_iter: 32/200
opt_iter: 33/200
opt_iter: 34/200
opt_iter: 35/200
opt_iter: 36/200
opt_iter: 37/200
opt_iter: 38/200
opt_iter: 39/200
opt_iter: 40/200
opt_iter: 41/200
opt_iter: 42/200
opt_iter: 43/200
opt_iter: 44/200
opt_iter: 45/200
opt_iter: 46/200
opt_iter: 47/200
opt_iter: 48/200
opt_iter: 49/200
opt_iter: 50/200
opt_iter: 51/200
opt_iter: 52/200
opt_iter: 53/200
opt_iter: 54/200
opt_iter: 55/200
opt_iter: 56/200
opt_iter: 57/200
opt_iter: 58/200
opt_iter: 59/200
opt_it

In [102]:
model1.make_params_df()

,noise,variance,lengthscale
0,1.000000,1.000000,0.500000
1,0.990050,1.010050,0.505025
2,0.980196,1.020198,0.510098
3,0.970437,1.030442,0.515217
4,0.960769,1.040782,0.520380
...,...,...,...
196,0.051696,4.073686,0.943412
197,0.050539,4.094551,0.942738
198,0.049404,4.115468,0.942022
199,0.048290,4.136438,0.941262


## SoR

In [98]:
model2 = Sparse_GP_regression_SoR(X, y, p_optimized=True)
mean2, cov2 = model2.predict(X_pred.clone())

# visualize(X_pred, f, mean2, cov2, p_inputs=model2.pseudo_inputs, title="SoR")

In [99]:
model2.optimize(iteration=200, learning_rate=0.01)
mean2_opt, cov2_opt = model2.predict(X_pred.clone())

# visualize(X_pred, f, mean2_opt, cov2_opt, p_inputs=model2.pseudo_inputs, title="Optimized SoR")

opt_iter: 1/200
opt_iter: 2/200
opt_iter: 3/200
opt_iter: 4/200
opt_iter: 5/200
opt_iter: 6/200
opt_iter: 7/200
opt_iter: 8/200
opt_iter: 9/200
opt_iter: 10/200
opt_iter: 11/200
opt_iter: 12/200
opt_iter: 13/200
opt_iter: 14/200
opt_iter: 15/200
opt_iter: 16/200
opt_iter: 17/200
opt_iter: 18/200
opt_iter: 19/200
opt_iter: 20/200
opt_iter: 21/200
opt_iter: 22/200
opt_iter: 23/200
opt_iter: 24/200
opt_iter: 25/200
opt_iter: 26/200
opt_iter: 27/200
opt_iter: 28/200
opt_iter: 29/200
opt_iter: 30/200
opt_iter: 31/200
opt_iter: 32/200
opt_iter: 33/200
opt_iter: 34/200
opt_iter: 35/200
opt_iter: 36/200
opt_iter: 37/200
opt_iter: 38/200
opt_iter: 39/200
opt_iter: 40/200
opt_iter: 41/200
opt_iter: 42/200
opt_iter: 43/200
opt_iter: 44/200
opt_iter: 45/200
opt_iter: 46/200
opt_iter: 47/200
opt_iter: 48/200
opt_iter: 49/200
opt_iter: 50/200
opt_iter: 51/200
opt_iter: 52/200
opt_iter: 53/200
opt_iter: 54/200
opt_iter: 55/200
opt_iter: 56/200
opt_iter: 57/200
opt_iter: 58/200
opt_iter: 59/200
opt_it

In [100]:
model2.make_params_df()

,noise,variance,lengthscale,u_1,u_2,u_3,u_4,u_5,u_6,u_7,u_8,u_9,u_10
0,1.000000,1.000000,0.500000,0.999731,1.774372,2.549012,3.323653,4.098293,4.872934,5.647574,6.422214,7.196855,7.971495
1,0.990050,1.010050,0.505025,1.009731,1.784372,2.559012,3.313653,4.108293,4.862934,5.637574,6.432214,7.186855,7.961495
2,0.980194,1.020200,0.510065,1.019704,1.794339,2.568933,3.314649,4.116527,4.853544,5.627633,6.440354,7.177133,7.951505
3,0.970430,1.030450,0.515095,1.029613,1.804274,2.578772,3.320132,4.121724,4.845182,5.617785,6.444156,7.167861,7.941534
4,0.960753,1.040801,0.520083,1.039410,1.814179,2.588522,3.327286,4.123969,4.838044,5.608054,6.443541,7.159102,7.931596
...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,0.042544,6.454843,0.447061,1.061071,1.987326,2.620075,3.333212,4.010362,4.653823,5.267927,6.167223,6.917839,7.707902
197,0.041558,6.512527,0.447141,1.061186,1.985551,2.619605,3.333238,4.010471,4.655941,5.268230,6.167650,6.920149,7.707936
198,0.040591,6.570703,0.447395,1.063178,1.982501,2.617110,3.335722,4.008265,4.659973,5.267832,6.166779,6.923382,7.707485
199,0.039644,6.629376,0.447289,1.061463,1.981761,2.617614,3.334538,4.009846,4.660091,5.268501,6.169617,6.922688,7.709701


## DTC

In [109]:
model3 = Sparse_GP_regression_DTC(X, y, p_optimized=True)
mean3, cov3 = model3.predict(X_pred.clone())

# visualize(X_pred, f, mean3, cov3, p_inputs=model3.pseudo_inputs, title="DTC")

In [110]:
model3.optimize(iteration=200, learning_rate=0.01)
mean3_opt, cov3_opt = model3.predict(X_pred.clone())

# visualize(X_pred, f, mean3_opt, cov3_opt, p_inputs=model3.pseudo_inputs, title="Optimized DTC")

opt_iter: 1/200
opt_iter: 2/200
opt_iter: 3/200
opt_iter: 4/200
opt_iter: 5/200
opt_iter: 6/200
opt_iter: 7/200
opt_iter: 8/200
opt_iter: 9/200
opt_iter: 10/200
opt_iter: 11/200
opt_iter: 12/200
opt_iter: 13/200
opt_iter: 14/200
opt_iter: 15/200
opt_iter: 16/200
opt_iter: 17/200
opt_iter: 18/200
opt_iter: 19/200
opt_iter: 20/200
opt_iter: 21/200
opt_iter: 22/200
opt_iter: 23/200
opt_iter: 24/200
opt_iter: 25/200
opt_iter: 26/200
opt_iter: 27/200
opt_iter: 28/200
opt_iter: 29/200
opt_iter: 30/200
opt_iter: 31/200
opt_iter: 32/200
opt_iter: 33/200
opt_iter: 34/200
opt_iter: 35/200
opt_iter: 36/200
opt_iter: 37/200
opt_iter: 38/200
opt_iter: 39/200
opt_iter: 40/200
opt_iter: 41/200
opt_iter: 42/200
opt_iter: 43/200
opt_iter: 44/200
opt_iter: 45/200
opt_iter: 46/200
opt_iter: 47/200
opt_iter: 48/200
opt_iter: 49/200
opt_iter: 50/200
opt_iter: 51/200
opt_iter: 52/200
opt_iter: 53/200
opt_iter: 54/200
opt_iter: 55/200
opt_iter: 56/200
opt_iter: 57/200
opt_iter: 58/200
opt_iter: 59/200
opt_it

In [111]:
model3.make_params_df()

,noise,variance,lengthscale,u_1,u_2,u_3,u_4,u_5,u_6,u_7,u_8,u_9,u_10
0,1.000000,1.000000,0.500000,0.999731,1.774372,2.549012,3.323653,4.098293,4.872934,5.647574,6.422214,7.196855,7.971495
1,0.990050,1.010050,0.505025,1.009731,1.784372,2.559012,3.313653,4.108293,4.862934,5.637574,6.432214,7.186855,7.961495
2,0.980194,1.020200,0.510065,1.019704,1.794339,2.568933,3.314649,4.116527,4.853544,5.627633,6.440354,7.177133,7.951505
3,0.970430,1.030450,0.515095,1.029613,1.804274,2.578772,3.320132,4.121724,4.845182,5.617785,6.444156,7.167861,7.941534
4,0.960753,1.040801,0.520083,1.039410,1.814179,2.588522,3.327286,4.123969,4.838044,5.608054,6.443541,7.159102,7.931596
...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,0.042544,6.454843,0.447061,1.061071,1.987326,2.620075,3.333212,4.010362,4.653823,5.267927,6.167223,6.917839,7.707902
197,0.041558,6.512527,0.447141,1.061186,1.985551,2.619605,3.333238,4.010471,4.655941,5.268230,6.167650,6.920149,7.707936
198,0.040591,6.570703,0.447395,1.063178,1.982501,2.617110,3.335722,4.008265,4.659973,5.267832,6.166779,6.923382,7.707485
199,0.039644,6.629376,0.447289,1.061463,1.981761,2.617614,3.334538,4.009846,4.660091,5.268501,6.169617,6.922688,7.709701


## FITC

In [113]:
model4 = Sparse_GP_regression_FITC(X, y, p_optimized=True)
mean4, cov4 = model4.predict(X_pred.clone())

# visualize(X_pred, f, mean4, cov4, p_inputs=model4.pseudo_inputs, title="FITC")

In [114]:
model4.optimize(iteration=200, learning_rate=0.01)
mean4_opt, cov4_opt = model4.predict(X_pred.clone())

# visualize(X_pred, f, mean4_opt, cov4_opt, p_inputs=model4.pseudo_inputs, title="Optimized FITC")

opt_iter: 1/200
opt_iter: 2/200
opt_iter: 3/200
opt_iter: 4/200
opt_iter: 5/200
opt_iter: 6/200
opt_iter: 7/200
opt_iter: 8/200
opt_iter: 9/200
opt_iter: 10/200
opt_iter: 11/200
opt_iter: 12/200
opt_iter: 13/200
opt_iter: 14/200
opt_iter: 15/200
opt_iter: 16/200
opt_iter: 17/200
opt_iter: 18/200
opt_iter: 19/200
opt_iter: 20/200
opt_iter: 21/200
opt_iter: 22/200
opt_iter: 23/200
opt_iter: 24/200
opt_iter: 25/200
opt_iter: 26/200
opt_iter: 27/200
opt_iter: 28/200
opt_iter: 29/200
opt_iter: 30/200
opt_iter: 31/200
opt_iter: 32/200
opt_iter: 33/200
opt_iter: 34/200
opt_iter: 35/200
opt_iter: 36/200
opt_iter: 37/200
opt_iter: 38/200
opt_iter: 39/200
opt_iter: 40/200
opt_iter: 41/200
opt_iter: 42/200
opt_iter: 43/200
opt_iter: 44/200
opt_iter: 45/200
opt_iter: 46/200
opt_iter: 47/200
opt_iter: 48/200
opt_iter: 49/200
opt_iter: 50/200
opt_iter: 51/200
opt_iter: 52/200
opt_iter: 53/200
opt_iter: 54/200
opt_iter: 55/200
opt_iter: 56/200
opt_iter: 57/200
opt_iter: 58/200
opt_iter: 59/200
opt_it

In [115]:
model4.make_params_df()

,noise,variance,lengthscale,u_1,u_2,u_3,u_4,u_5,u_6,u_7,u_8,u_9,u_10
0,1.000000,1.000000,0.500000,0.999731,1.774372,2.549012,3.323653,4.098293,4.872934,5.647574,6.422214,7.196855,7.971495
1,0.990050,1.010050,0.505025,1.009731,1.784372,2.559012,3.313653,4.108293,4.862934,5.637574,6.432214,7.186855,7.961495
2,0.980194,1.020200,0.510065,1.019704,1.794339,2.568933,3.314649,4.116527,4.853544,5.627633,6.440354,7.177133,7.951505
3,0.970430,1.030450,0.515095,1.029613,1.804274,2.578772,3.320132,4.121724,4.845182,5.617785,6.444156,7.167861,7.941534
4,0.960753,1.040801,0.520083,1.039410,1.814179,2.588522,3.327286,4.123969,4.838044,5.608054,6.443541,7.159102,7.931596
...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,0.042544,6.454843,0.447061,1.061071,1.987326,2.620075,3.333212,4.010362,4.653823,5.267927,6.167223,6.917839,7.707902
197,0.041558,6.512527,0.447141,1.061186,1.985551,2.619605,3.333238,4.010471,4.655941,5.268230,6.167650,6.920149,7.707936
198,0.040591,6.570703,0.447395,1.063178,1.982501,2.617110,3.335722,4.008265,4.659973,5.267832,6.166779,6.923382,7.707485
199,0.039644,6.629376,0.447289,1.061463,1.981761,2.617614,3.334538,4.009846,4.660091,5.268501,6.169617,6.922688,7.709701
